<a href="https://colab.research.google.com/github/Jarde01/ml-tools-and-examples/blob/master/troll_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install wandb pytorch-transformers

     |████████████████████████████████| 1.3MB 3.4MB/s 
     |████████████████████████████████| 143kB 51.7MB/s 
     |████████████████████████████████| 235kB 46.2MB/s 
     |████████████████████████████████| 81kB 33.0MB/s 
     |████████████████████████████████| 102kB 32.9MB/s 
     |████████████████████████████████| 92kB 28.8MB/s 
     |████████████████████████████████| 460kB 50.1MB/s 
     |████████████████████████████████| 655kB 50.4MB/s 
     |████████████████████████████████| 1.0MB 40.6MB/s 
     |████████████████████████████████| 256kB 61.3MB/s 
     |████████████████████████████████| 71kB 32.1MB/s 
     |████████████████████████████████| 184kB 57.3MB/s 
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5540 sha256=b6f95219951b87021461f9e4c9ed1ab8e67609d8fc100e86f6aa91107db290e4
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl s

In [0]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, Dense, Dropout, Activation, Embedding, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras import backend as K
from keras.preprocessing.text import *
from keras.preprocessing import *
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, LabelEncoder

Using TensorFlow backend.


In [4]:
!wandb login 2295caae2022fe6a431a755f74d1e9e8605848eb
import wandb
from wandb.keras import WandbCallback
wandb.init(project="troll-detection")

Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


W&B Run: https://app.wandb.ai/jeremy/troll-detection/runs/4iosw9h1

In [0]:
import torch
from pytorch_transformers import *

In [6]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

100%|██████████| 440473133/440473133 [00:06<00:00, 69042634.16B/s]


In [7]:
input_ids = torch.tensor([bert_tokenizer.encode("some words here ./ and some punctuation..")])
# last_hidden_states = bert_model(input_ids)[0]  # Models outputs are now tuples
last_hidden_states = bert_model(input_ids)[0]  # Models outputs are now tuples
vectors = last_hidden_states.detach().numpy()[0]
vectors.shape

(12, 768)

In [0]:
def pad_sentence(sentence:str, length:int=15, fill:str='word'):
  tokens = sentence.split()
  num_tokens = len(tokens)
  
  if num_tokens < length:
    for i in range(0, length-num_tokens):
      tokens.append(fill)
  else:
    tokens = tokens[:length]
#   assert(len(sentence)) == length
  
  return " ".join(tokens)

## Preprocess the training data

In [10]:
df = pd.read_json("/content/Dataset for Detection of Cyber-Trolls.json", lines=True)
df.head()

,annotation,content,extras
0,"{'notes': '', 'label': ['1']}",Get fucking real dude.,NaN
1,"{'notes': '', 'label': ['1']}",She is as dirty as they come and that crook ...,NaN
2,"{'notes': '', 'label': ['1']}",why did you fuck it up. I could do it all day...,NaN
3,"{'notes': '', 'label': ['1']}",Dude they dont finish enclosing the fucking s...,NaN
4,"{'notes': '', 'label': ['1']}",WTF are you talking about Men? No men thats n...,NaN


In [0]:
labels = df['annotation']
labels = labels.apply(lambda x: x['label'][0])

In [0]:
# labels_processed = labels.tolist()
# le = LabelEncoder()
# labels_processed = le.fit_transform(labels_processed)

from keras.utils import to_categorical
labels_processed = to_categorical(labels.tolist())

In [0]:
from sklearn.preprocessing import LabelEncoder
inputs = df['content'].values

In [0]:
def bert_layer(x):
  input_ids = torch.tensor([bert_tokenizer.encode(x)])
  last_hidden_states = bert_model(input_ids)[0]
  final = last_hidden_states.detach().numpy()
  return final

# processed = fixed_sent.apply(lambda x: bert_layer(x))

In [0]:
from keras.preprocessing import *

tok = text.Tokenizer()
tok.fit_on_texts(inputs)

sequences = tok.texts_to_sequences(inputs)

In [0]:
padded_inputs = sequence.pad_sequences(sequences, maxlen=50)

In [155]:
padded_inputs.shape

(20001, 50)

In [156]:
num_words = len(tok.word_counts)

print(padded_inputs[:5])
print(labels_processed[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   37   38  293  151]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0  105   10
    62  818   62   48  188    7    6 4897 4898    3 4899   31   30   38
  3333   61    4  787  108 3904  175   27]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   80   92    2   25    8   45    1  121
    20    8   41  127   54  622   20    8   50    2   22   91  551 2111
    17  584    5 4900  861    4  450  109]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  151   48  113 1258 4901    3   38 1939    1    9  424 3905 1078
   438 

In [0]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(padded_inputs, labels_processed, test_size=0.2)

In [168]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(16000, 50)
(4001, 50)
(16000, 2)
(4001, 2)


In [0]:
import tensorflow as tf

def get_model(num_words, embed_out=128, dense_1_nodes=32, dropout_1=0.5):

# keras.layers.InputLayer(..., ragged=True)
  
#   inputs = Input(ragged=true) 
# #   inputs = Lambda(lambda x: bert_layer(x), output_shape=(768,))
#   x = Lambda(lambda x: bert_layer(x), output_shape=(768,))(inputs)

# #   inputs = Lambda(lambda x: bert_layer(x), input_shape=(30,), dtype=tf.string)

#   x = Dense(64, activation='relu')(inputs)
#   outputs = Dense(2, activation='softmax')(x)  
  #   model = Model(inputs=inputs, outputs=outputs)

  
  # This is the sequential model version
  model = Sequential()
  model.add(Embedding(num_words, embed_out))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(dense_1_nodes, activation='relu', name='dense_1'))
  model.add(Dropout(dropout_1))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
  return model            

In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
import random

def run_trials_single():
  import random

  run_name = "" # what are you testing with this run
  notes = ""

  #hyperparams
  hyperparams = dict(
    embed_out =random.randint(10,500),
    dense_1_nodes = random.randint(10,128),
    dropout_1=random.uniform(0.0, 0.9),
  )

  config= {
      "epochs": 50,
  #     "num_classes":num_classes,
      "num_words": num_words,
      "batch_size":1024,
      **hyperparams
  }

  model = get_model(num_words, **hyperparams)

  wandb.init(project="troll-detection ", name=run_name, config=config, notes=notes)

  history = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], 
            validation_split=0.2, callbacks=[WandbCallback()])

  scores = model.evaluate(x_test, y_test)
  wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
  print("\n*** Finished! ***")

In [178]:
%%wandb
for x in range(0,100):
  run_trials_single()

12800/12800 [==============================] - 0s 12us/step - loss: 0.0099 - acc: 0.9958 - f1_m: 0.9958 - precision_m: 0.9958 - recall_m: 0.9958 - val_loss: 0.4812 - val_acc: 0.8697 - val_f1_m: 0.8697 - val_precision_m: 0.8697 - val_recall_m: 0.8697
Epoch 40/50
12800/12800 [==============================] - 0s 12us/step - loss: 0.0101 - acc: 0.9957 - f1_m: 0.9957 - precision_m: 0.9957 - recall_m: 0.9957 - val_loss: 0.5046 - val_acc: 0.8644 - val_f1_m: 0.8644 - val_precision_m: 0.8644 - val_recall_m: 0.8644
Epoch 41/50
12800/12800 [==============================] - 0s 12us/step - loss: 0.0094 - acc: 0.9956 - f1_m: 0.9956 - precision_m: 0.9956 - recall_m: 0.9956 - val_loss: 0.4947 - val_acc: 0.8675 - val_f1_m: 0.8675 - val_precision_m: 0.8675 - val_recall_m: 0.8675
Epoch 42/50
12800/12800 [==============================] - 0s 13us/step - loss: 0.0099 - acc: 0.9952 - f1_m: 0.9952 - precision_m: 0.9952 - recall_m: 0.9952 - val_loss: 0.5058 - val_acc: 0.8656 - val_f1_m: 0.8656 - val_precisi

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 1s 83us/step - loss: 0.6754 - acc: 0.5974 - f1_m: 0.5974 - precision_m: 0.5974 - recall_m: 0.5974 - val_loss: 0.6675 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 10us/step - loss: 0.6610 - acc: 0.6067 - f1_m: 0.6067 - precision_m: 0.6067 - recall_m: 0.6067 - val_loss: 0.6533 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 11us/step - loss: 0.6375 - acc: 0.6067 - f1_m: 0.6067 - precision_m: 0.6067 - recall_m: 0.6067 - val_loss: 0.6258 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 4/50
12800/12800 [==============================] - 0s 10us/step - loss: 0.5893 - acc: 0.6255 - f1_m: 0.6255 - precision_m: 0.6255 - recall_m: 0.6255 - val_loss: 

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 1s 95us/step - loss: 0.6754 - acc: 0.6027 - f1_m: 0.6025 - precision_m: 0.6026 - recall_m: 0.6025 - val_loss: 0.6626 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 15us/step - loss: 0.6579 - acc: 0.6067 - f1_m: 0.6067 - precision_m: 0.6067 - recall_m: 0.6067 - val_loss: 0.6486 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 14us/step - loss: 0.6343 - acc: 0.6068 - f1_m: 0.6068 - precision_m: 0.6068 - recall_m: 0.6068 - val_loss: 0.6267 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 4/50
12800/12800 [==============================] - 0s 13us/step - loss: 0.6020 - acc: 0.6072 - f1_m: 0.6072 - precision_m: 0.6072 - recall_m: 0.6072 - val_loss: 

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 1s 115us/step - loss: 0.6688 - acc: 0.5989 - f1_m: 0.5989 - precision_m: 0.5989 - recall_m: 0.5989 - val_loss: 0.6536 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 18us/step - loss: 0.6355 - acc: 0.6117 - f1_m: 0.6117 - precision_m: 0.6117 - recall_m: 0.6117 - val_loss: 0.6194 - val_acc: 0.6128 - val_f1_m: 0.6128 - val_precision_m: 0.6128 - val_recall_m: 0.6128
Epoch 3/50
12800/12800 [==============================] - 0s 18us/step - loss: 0.5760 - acc: 0.6956 - f1_m: 0.6956 - precision_m: 0.6956 - recall_m: 0.6956 - val_loss: 0.5588 - val_acc: 0.7231 - val_f1_m: 0.7231 - val_precision_m: 0.7231 - val_recall_m: 0.7231
Epoch 4/50
12800/12800 [==============================] - 0s 16us/step - loss: 0.4854 - acc: 0.8059 - f1_m: 0.8059 - precision_m: 0.8059 - recall_m: 0.8059 - val_loss:

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 1s 93us/step - loss: 0.6721 - acc: 0.6059 - f1_m: 0.6059 - precision_m: 0.6059 - recall_m: 0.6059 - val_loss: 0.6637 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 10us/step - loss: 0.6580 - acc: 0.6068 - f1_m: 0.6068 - precision_m: 0.6068 - recall_m: 0.6068 - val_loss: 0.6490 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 8us/step - loss: 0.6331 - acc: 0.6082 - f1_m: 0.6082 - precision_m: 0.6082 - recall_m: 0.6082 - val_loss: 0.6200 - val_acc: 0.6141 - val_f1_m: 0.6141 - val_precision_m: 0.6141 - val_recall_m: 0.6141
Epoch 4/50
12800/12800 [==============================] - 0s 9us/step - loss: 0.5839 - acc: 0.6800 - f1_m: 0.6800 - precision_m: 0.6800 - recall_m: 0.6800 - val_loss: 0.

...

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 10s 759us/step - loss: 0.6782 - acc: 0.5843 - f1_m: 0.5843 - precision_m: 0.5845 - recall_m: 0.5840 - val_loss: 0.6637 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 24us/step - loss: 0.6616 - acc: 0.6086 - f1_m: 0.6086 - precision_m: 0.6086 - recall_m: 0.6086 - val_loss: 0.6500 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 20us/step - loss: 0.6391 - acc: 0.6180 - f1_m: 0.6180 - precision_m: 0.6180 - recall_m: 0.6180 - val_loss: 0.6259 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 4/50
12800/12800 [==============================] - 0s 21us/step - loss: 0.6109 - acc: 0.6387 - f1_m: 0.6387 - precision_m: 0.6387 - recall_m: 0.6387 - val_loss

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 10s 775us/step - loss: 0.6784 - acc: 0.5882 - f1_m: 0.5879 - precision_m: 0.5881 - recall_m: 0.5878 - val_loss: 0.6623 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 25us/step - loss: 0.6580 - acc: 0.6076 - f1_m: 0.6076 - precision_m: 0.6076 - recall_m: 0.6076 - val_loss: 0.6452 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 25us/step - loss: 0.6339 - acc: 0.6176 - f1_m: 0.6176 - precision_m: 0.6176 - recall_m: 0.6176 - val_loss: 0.6188 - val_acc: 0.6169 - val_f1_m: 0.6169 - val_precision_m: 0.6169 - val_recall_m: 0.6169
Epoch 4/50
12800/12800 [==============================] - 0s 21us/step - loss: 0.5966 - acc: 0.6506 - f1_m: 0.6506 - precision_m: 0.6506 - recall_m: 0.6506 - val_loss

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 10s 778us/step - loss: 0.6696 - acc: 0.6011 - f1_m: 0.6011 - precision_m: 0.6011 - recall_m: 0.6011 - val_loss: 0.6559 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 26us/step - loss: 0.6323 - acc: 0.6080 - f1_m: 0.6080 - precision_m: 0.6080 - recall_m: 0.6080 - val_loss: 0.6139 - val_acc: 0.6412 - val_f1_m: 0.6412 - val_precision_m: 0.6412 - val_recall_m: 0.6412
Epoch 3/50
12800/12800 [==============================] - 0s 25us/step - loss: 0.5562 - acc: 0.7175 - f1_m: 0.7175 - precision_m: 0.7175 - recall_m: 0.7175 - val_loss: 0.5385 - val_acc: 0.7538 - val_f1_m: 0.7537 - val_precision_m: 0.7538 - val_recall_m: 0.7538
Epoch 4/50
12800/12800 [==============================] - 0s 23us/step - loss: 0.4410 - acc: 0.8416 - f1_m: 0.8416 - precision_m: 0.8416 - recall_m: 0.8416 - val_loss

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 10s 784us/step - loss: 0.6739 - acc: 0.5989 - f1_m: 0.5989 - precision_m: 0.5989 - recall_m: 0.5989 - val_loss: 0.6610 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 24us/step - loss: 0.6546 - acc: 0.6068 - f1_m: 0.6068 - precision_m: 0.6068 - recall_m: 0.6068 - val_loss: 0.6426 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 22us/step - loss: 0.6236 - acc: 0.6104 - f1_m: 0.6104 - precision_m: 0.6104 - recall_m: 0.6104 - val_loss: 0.6092 - val_acc: 0.6131 - val_f1_m: 0.6131 - val_precision_m: 0.6131 - val_recall_m: 0.6131
Epoch 4/50
12800/12800 [==============================] - 0s 23us/step - loss: 0.5726 - acc: 0.6660 - f1_m: 0.6660 - precision_m: 0.6660 - recall_m: 0.6660 - val_loss

Train on 12800 samples, validate on 3200 samples
Epoch 1/50
12800/12800 [==============================] - 10s 798us/step - loss: 0.6760 - acc: 0.5929 - f1_m: 0.5929 - precision_m: 0.5929 - recall_m: 0.5929 - val_loss: 0.6612 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 2/50
12800/12800 [==============================] - 0s 26us/step - loss: 0.6517 - acc: 0.6070 - f1_m: 0.6070 - precision_m: 0.6070 - recall_m: 0.6070 - val_loss: 0.6393 - val_acc: 0.6116 - val_f1_m: 0.6116 - val_precision_m: 0.6116 - val_recall_m: 0.6116
Epoch 3/50
12800/12800 [==============================] - 0s 22us/step - loss: 0.6109 - acc: 0.6310 - f1_m: 0.6310 - precision_m: 0.6310 - recall_m: 0.6310 - val_loss: 0.5927 - val_acc: 0.6797 - val_f1_m: 0.6797 - val_precision_m: 0.6797 - val_recall_m: 0.6797
Epoch 4/50
12800/12800 [==============================] - 0s 26us/step - loss: 0.5356 - acc: 0.7445 - f1_m: 0.7445 - precision_m: 0.7445 - recall_m: 0.7445 - val_loss